# Getting Coverage

In the [previous chapter](Mutation_Fuzzing.ipynb), we sketched how to use the coverage of _implemented functionality_ as guidance for test generation.  In this chapter, we show how to measure which parts of a program are actually executed during a test run, and how to use this _coverage_ for guiding test generation.

**Prerequisites**

* Getting coverage alone only requires basic understanding of how a program is executed.
* Leveraging coverage for guiding tests builds upon [Mutation-based fuzzing](Mutation_Fuzzing.ipynb).

## Tracing Executions

As discussed in the [previous chapter](Mutation_Fuzzing.ipynb), generating tests without any knowledge about the program to be tested will not only result in very many invalid tests, but also is not likely to cover a lot of the program's functionality.  To this end, we introduce a general mechanism to _trace_ the execution of a program, telling us exactly which program code was executed, and more.

### Tracing Executions in Python

In most programming languages, it is rather difficult to set up programs such that one can trace their execution.  Not so in Python.  The function `sys.settrace(f)` allows to define a function `f()` that is called for each and every line executed.  Even better, it gets access to the current function and its name, current variable contents, and more.  It is thus an ideal tool for _dynamic analysis_ – that is, the analysis of what actually happens during an execution.

Let us illustrate `sys.settrace()` in practice.  To this end, we make use of the `my_sqrt()` function from our [Introduction to Testing](Intro_Testing.ipynb):

In [1]:
import gstbook

In [2]:
%%capture
from Intro_Testing import my_sqrt

In [3]:
my_sqrt(2)

1.414213562373095

To track how the execution proceeds through `my_sqrt()`, we make use of `sys.settrace()`.  First, we define the _tracing function_ that will be called for each line.  It has three parameters: 

* the current _frame_, allowing access to the current location and variables;
* the current _event_, including `"line"` (a new line has been reached) or `"call"` (a function is being called)
* an additional _argument_ for some events.

We use the tracing function for simply reporting the current line executed, which we access through the `frame` argument.

In [4]:
def traceit(frame, event, arg):
    if event == "line":
        function_name = frame.f_code.co_name
        lineno = frame.f_lineno
        print(function_name + "(), line", lineno)
    return traceit

We can switch tracing on and off with `sys.settrace()`:

In [5]:
import sys

def my_sqrt_traced(x):
    sys.settrace(traceit)
    my_sqrt(x)
    sys.settrace(None)

When we compute $\sqrt(4)$, we can now see how the execution progresses through `my_sqrt()`.  Only one loop iteration is required:

In [12]:
my_sqrt_traced(4)

my_sqrt(), line 3
my_sqrt(), line 4
my_sqrt(), line 5
my_sqrt(), line 6
my_sqrt(), line 7
my_sqrt(), line 5
my_sqrt(), line 8


Computing $\sqrt{2}$, though, requires six loop iterations:

In [13]:
my_sqrt_traced(2)

my_sqrt(), line 3
my_sqrt(), line 4
my_sqrt(), line 5
my_sqrt(), line 6
my_sqrt(), line 7
my_sqrt(), line 5
my_sqrt(), line 6
my_sqrt(), line 7
my_sqrt(), line 5
my_sqrt(), line 6
my_sqrt(), line 7
my_sqrt(), line 5
my_sqrt(), line 6
my_sqrt(), line 7
my_sqrt(), line 5
my_sqrt(), line 6
my_sqrt(), line 7
my_sqrt(), line 5
my_sqrt(), line 6
my_sqrt(), line 7
my_sqrt(), line 5
my_sqrt(), line 8


We can now easily extend the `traceit()` function to not only print out the lines executed, but also to _save_ them, such that we can access them later.

### A Coverage Package


In [20]:
import sys

class Coverage(object):
    # Trace function
    def traceit(self, frame, event, arg):
        if self.original_trace_function is not None:
            self.original_trace_function(frame, event, arg)
            
        if event == "line":
            function_name = frame.f_code.co_name
            lineno = frame.f_lineno
            self._trace.append((function_name, lineno))
            
        return self.traceit
    
    def __init__(self):
        self._trace = []
    
    # Start of `with` block
    def __enter__(self):
        self.original_trace_function = sys.gettrace()
        sys.settrace(self.traceit)
        return self

    # End of `with` block
    def __exit__(self, exc_type, exc_value, tb):
        sys.settrace(self.original_trace_function)

    def trace(self):
        """The list of executed lines, as (function_name, line_number) pairs"""
        return self._trace
    
    def coverage(self):
        """The set of executed lines, as (function_name, line_number) pairs"""
        return set(self.trace())

with Coverage() as cov:
    my_sqrt(2)

print(cov.coverage())

{('my_sqrt', 8), ('my_sqrt', 6), ('my_sqrt', 7), ('my_sqrt', 4), ('my_sqrt', 5), ('my_sqrt', 3), ('__exit__', 27)}


## Measuring Coverage

We can use coverage to assess the _effectiveness_ of various tests.  Generally spoken, if a statement is never executed (covered) during testing, this means that an error in this statement cannot be triggered either.  Good testing thus mandates that at the very least, each executable statement in the code be covered at least once.

\todo{Our previous examples (sqrt and expr) don't work that well for discussing coverage.  Introduce the cgi_decode function here?  Apply mutation-based testing on it?}

Beyond this simple _statement coverage_, more advanced measures exist that also take into account branches, paths, or the number of loops taken; see the [exercises](#Exercises) for details.

## Leveraging Coverage

\todo{Use coverage metrics to guide mutation-based testing, by keeping those individuals in the population that have different coverage.}

## Next Steps

_Link to subsequent chapters (notebooks) here, as in:_

* [use _mutations_ on existing inputs to get more valid inputs](Mutation_Fuzzing.ipynb)
* [use _grammars_ (i.e., a specification of the input format) to get even more valid inputs](Grammars.ipynb)
* [reduce _failing inputs_ for efficient debugging](Reducing.ipynb)


## Exercises

_Close the chapter with a few exercises such that people have things to do.  Use the Jupyter `Exercise2` nbextension to add solutions that can be interactively viewed or hidden.  (Alternatively, just copy the exercise and solution cells below with their metadata.)  We will set up things such that solutions do not appear in the PDF and HTML formats._

### Exercise 1

_Text of the exercise_

_Solution for the exercise_